# Exploring German eBay car sales

Original data is hosted on kaggle [here](https://www.kaggle.com/orgesleka/used-cars-database/data) but we are using a version that has been downsampled to 50,000 rows. As this is a teaching project it has also been deliberately "dirtied". 

The data dictionary is below:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

print(autos.info())
print('-' * 100)
print(autos.isnull().sum())
print('-' * 100)
print(autos.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

## Issues with the data

`dateCrawled, price, odometer, dateCreated, lastSeen` are all the wrong type. 

`vehicleType 5095, gearbox 2680, model 2758, fuelType 4482, notRepairedDamage 9829` - null entries. 

Also noticed on row 4 there is a vehicle with 0 as it's entry for 'powerPS' so while not null it is effectively a null entry. 

The columns are not in snake case. 

`price` and `odometer` will need the non numeric charachers removing and convering into intergers. 

## Fixing the column names

We are going to fix the column names by converting them to snakecase so they are easier to use throughout the rest of this notebook. 

In [2]:
print(autos.columns)
print('-' * 75)

cols = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
       'last_seen']

autos.columns = cols

print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')
---------------------------------------------------------------------------
Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


## We are now going to explore the data

We are going to do a high level pass over the data in all the columns so we can see what we need to keep, what we need to clean and how. 

In [3]:
print(autos.describe(include = 'all'))

               date_crawled         name  seller offer_type  price abtest  \
count                 50000        50000   50000      50000  50000  50000   
unique                48213        38754       2          2   2357      2   
top     2016-03-30 17:37:35  Ford_Fiesta  privat    Angebot     $0   test   
freq                      3           78   49999      49999   1421  25756   
mean                    NaN          NaN     NaN        NaN    NaN    NaN   
std                     NaN          NaN     NaN        NaN    NaN    NaN   
min                     NaN          NaN     NaN        NaN    NaN    NaN   
25%                     NaN          NaN     NaN        NaN    NaN    NaN   
50%                     NaN          NaN     NaN        NaN    NaN    NaN   
75%                     NaN          NaN     NaN        NaN    NaN    NaN   
max                     NaN          NaN     NaN        NaN    NaN    NaN   

       vehicle_type  registration_year  gearbox      power_ps  model  \
cou

## Columns to be dropped.

`seller` and `offer_type` both have only 2 unique vaules and both have 49999 of one value and a single of the other. This will not help our analysis but keeping this values. 

## Cleaning `price` and `odometer`

We are going now clean the `price` and `odometer` columns by removing the non numeric's and converting them into intergers. We are also going relabel `odometer` to `odometer_km` so if we come back to the data later we will be able to work out what we have done. 

In [4]:
print(autos["price"].unique())
print('-' * 10)
print(autos["odometer"].unique())
print('-' * 75)
autos["price"] = autos["price"].str.replace("$","")
autos["price"] = autos["price"].str.replace(",", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["price"] = autos["price"].apply(pd.to_numeric)
autos["odometer"] = autos["odometer"].apply(pd.to_numeric)
print(autos["price"].unique())
print('-' * 10)
print(autos["odometer"].unique())
autos.rename(columns={'odometer':'odometer_km'}, inplace = True)

['$5,000' '$8,500' '$8,990' ... '$385' '$22,200' '$16,995']
----------
['150,000km' '70,000km' '50,000km' '80,000km' '10,000km' '30,000km'
 '125,000km' '90,000km' '20,000km' '60,000km' '5,000km' '100,000km'
 '40,000km']
---------------------------------------------------------------------------
[ 5000  8500  8990 ...   385 22200 16995]
----------
[150000  70000  50000  80000  10000  30000 125000  90000  20000  60000
   5000 100000  40000]


## Exploring `price` and `odometer_km` further

We are now going to explore these columns further now the are numeric. 

In [5]:
print("Price's shape is : ",autos["price"].unique().shape)
print('-' * 10)
print("Odometer's shape is : ",autos["odometer_km"].unique().shape)
print('-' * 75)
print(autos["price"].value_counts().sort_index(ascending=True).head(55))
print('-' * 10)
print(autos["odometer_km"].value_counts().sort_index(ascending=True).head())
print('-' * 75)
print(autos["price"].value_counts().sort_index(ascending=False).head(15))
print('-' * 10)
print(autos["odometer_km"].value_counts().sort_index(ascending=False).head())

Price's shape is :  (2357,)
----------
Odometer's shape is :  (13,)
---------------------------------------------------------------------------
0      1421
1       156
2         3
3         1
5         2
8         1
9         1
10        7
11        2
12        3
13        2
14        1
15        2
17        3
18        1
20        4
25        5
29        1
30        7
35        1
40        6
45        4
47        1
49        4
50       49
55        2
59        1
60        9
65        5
66        1
70       10
75        5
79        1
80       15
89        1
90        5
99       19
100     134
110       3
111       2
115       2
117       1
120      39
122       1
125       8
129       1
130      15
135       1
139       1
140       9
145       2
149       7
150     224
156       2
160       8
Name: price, dtype: int64
----------
5000     967
10000    264
20000    784
30000    789
40000    819
Name: odometer_km, dtype: int64
--------------------------------------------------------------

## Removal of `price` values over 350,000 and under 100

The top 10 price values are not only outliers, they are likely false adverts. While some vares are worth a million they are highly unlikely to be sold on eBay. AFAIK there has never been a car sold for nearly 100 million. 

All the 'free' cars and very low value cars are very likely to be people trying to game the 'Sort by price' on the website. 

In [6]:
print(autos["price"].describe())
autos  = autos[autos["price"].between( 100, 350000)]
print('-' * 25)
print(autos["price"].describe())
print('-' * 25)
print(autos.describe(include = 'all'))

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
-------------------------
count     48224.000000
mean       5930.371433
std        9078.372762
min         100.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      350000.000000
Name: price, dtype: float64
-------------------------
               date_crawled         name  seller offer_type          price  \
count                 48224        48224   48224      48224   48224.000000   
unique                46571        37162       2          1            NaN   
top     2016-03-29 23:42:13  Ford_Fiesta  privat    Angebot            NaN   
freq                      3           76   48223      48224            NaN   
mean                    NaN          NaN     NaN        NaN    5930.371433   
std                     NaN          NaN     NaN        NaN    9

## Date columns exploring

We are going to explore the 5 different date columns.

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

`registration_month` and `registration_year` are going to be simple as they are integers but currently `date_crawled`, `last_seen` and `ad_created` are strings so will require a bit more work.


In [11]:
print(autos[['date_crawled','ad_created','last_seen']][0:5])
print('-' * 25)
print(autos['date_crawled'].str[:10].head())
print('-' * 25)
print((autos['date_crawled']
       .str[:10].value_counts(normalize = True, dropna = False)
      .sort_index() * 100))
print('-' * 25)
print((autos['ad_created']
       .str[:10].value_counts(normalize = True, dropna = False)
      .sort_index() * 100))
print('-' * 25)
print((autos['last_seen']
       .str[:10].value_counts(normalize = True, dropna = False)
      .sort_index() * 100))
print('-' * 25)
print((autos['ad_created']
       .str[:10].value_counts(normalize = True, dropna = False)
      .sort_index().head(42).sum() * 100))
print('-' * 25)
print(autos["registration_year"].describe())

          date_crawled           ad_created            last_seen
0  2016-03-26 17:47:46  2016-03-26 00:00:00  2016-04-06 06:45:54
1  2016-04-04 13:38:56  2016-04-04 00:00:00  2016-04-06 14:45:08
2  2016-03-26 18:57:24  2016-03-26 00:00:00  2016-04-06 20:15:37
3  2016-03-12 16:58:10  2016-03-12 00:00:00  2016-03-15 03:16:28
4  2016-04-01 14:38:50  2016-04-01 00:00:00  2016-04-01 14:38:50
-------------------------
0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: date_crawled, dtype: object
-------------------------
2016-03-05    2.536082
2016-03-06    1.403865
2016-03-07    3.606088
2016-03-08    3.317850
2016-03-09    3.301261
2016-03-10    3.228683
2016-03-11    3.259788
2016-03-12    3.691108
2016-03-13    1.567684
2016-03-14    3.666224
2016-03-15    3.431901
2016-03-16    2.946666
2016-03-17    3.149884
2016-03-18    1.289814
2016-03-19    3.473374
2016-03-20    3.780275
2016-03-21    3.720139
2016-03-22    3.288819
2016-03-23    3.228683
2016-03

So we have found some of the cars have been either been on the site for 9 months or there was an error in the scrapping of the site. 

While I would expect a large percentage for the `last_seen` field on the last day of crawling. There is a large percentage for two days before hand (April 6th and 7th) which is interesting.

Also apparently some of the cars are well over a 1000 years old and some are from the future, lets hope the fly.

## Cleaning `registration_year`

We are going to drop any rows that are pre 1900 as that is pre true mass production of cars. While Karl Benz is said to have built the first production car in 1885, these are highly unlikely to come up for sale on eBay and there were so few made that dates before 1900 are more likely to be a mistake rather than one of the few specimens that still exist.

We are also going to remove the cars that were produced after 2016 as time travel is so unlikely in car sales it is a safe assumption to make. 

In [18]:
autos  = autos[autos["registration_year"].between( 1900, 2016)]
print('-' * 25)
print((autos["registration_year"].value_counts(normalize = True)
       .sort_values(ascending=False).head(10)))
print('-' * 25)
print((autos["registration_year"].value_counts(normalize = True)
       .sort_values(ascending=False).head(10).sum() * 100))

2000    0.066966
2005    0.062802
1999    0.062112
2004    0.058228
2003    0.058099
2006    0.057560
2001    0.056718
2002    0.053439
1998    0.050483
2007    0.049038
2008    0.047679
2009    0.044874
1997    0.041530
2011    0.034907
2010    0.034238
1996    0.029233
2012    0.028219
2016    0.025932
1995    0.025738
2013    0.017281
2014    0.014282
1994    0.013505
1993    0.009061
2015    0.008198
1992    0.007918
1991    0.007292
1990    0.007163
1989    0.003689
1988    0.002869
1985    0.002006
          ...   
1966    0.000475
1976    0.000453
1969    0.000410
1975    0.000388
1965    0.000367
1964    0.000259
1963    0.000173
1961    0.000129
1959    0.000129
1962    0.000086
1956    0.000086
1937    0.000086
1958    0.000086
1955    0.000043
1954    0.000043
1957    0.000043
1951    0.000043
1910    0.000043
1934    0.000043
1941    0.000043
1927    0.000022
1929    0.000022
1950    0.000022
1931    0.000022
1948    0.000022
1938    0.000022
1939    0.000022
1953    0.0000

Over 57% of all the cars available in the dataset were registered between 1998 and 2007.

## Exporing the data within the `brand` column

At the end of the below code block we are going to produce the mean price for a selection of brands.

During the exploration of the data I will decide on wether to use top `x` brands or brands over a certain percentage. An explantation over my reasoning will be provided after. 

In [28]:
print(autos["brand"].value_counts())
print('-' * 25)
print(autos["brand"].value_counts(normalize = True) * 100)
print('-' * 25)
print(autos["brand"].value_counts(normalize = True).head(20).sum() * 100)

volkswagen        9799
bmw               5107
opel              4971
mercedes_benz     4480
audi              4022
ford              3237
renault           2182
peugeot           1384
fiat              1187
seat               846
skoda              761
nissan             711
mazda              706
smart              658
citroen            651
toyota             593
hyundai            464
sonstige_autos     442
volvo              423
mini               408
mitsubishi         379
honda              365
kia                328
alfa_romeo         309
porsche            279
suzuki             275
chevrolet          263
chrysler           163
dacia              123
daihatsu           116
jeep               106
subaru              98
land_rover          98
saab                77
jaguar              71
daewoo              69
trabant             63
rover               62
lancia              49
lada                27
Name: brand, dtype: int64
-------------------------
volkswagen        21.140404


The top 20 represent over 92% of all the data in the dataset which is why I used that for producing our mean price aggregate.

Unsurprisingly the top 5 brands are German (if not all with German owners)

In [42]:
top_brands = autos["brand"].value_counts(normalize = True).head(20).index

brand_mean_price = {}

def make_mean(whatlist, dataset, cola, colb):
    output = {}
    for row in whatlist:
        brand = dataset[dataset[cola] == row]
        mean = brand[colb].mean()
        output[row] = mean
    return output

brand_mean_price = make_mean(top_brands, autos, 'brand', 'price')

print('The mean prices for the selected brands are below : ')
print('\n')
for key in brand_mean_price:
    print('-' * 5)
    print(("{}'s mean price in the dataset is : {}"
           .format(key.capitalize(),round(brand_mean_price[key]))))
    
print('-' * 75)

The mean prices for the selected brands are below : 


-----
Seat's mean price in the dataset is : 4433.0
-----
Sonstige_autos's mean price in the dataset is : 12785.0
-----
Mazda's mean price in the dataset is : 4130.0
-----
Ford's mean price in the dataset is : 3779.0
-----
Nissan's mean price in the dataset is : 4757.0
-----
Volkswagen's mean price in the dataset is : 5437.0
-----
Mercedes_benz's mean price in the dataset is : 8673.0
-----
Audi's mean price in the dataset is : 9381.0
-----
Citroen's mean price in the dataset is : 3796.0
-----
Opel's mean price in the dataset is : 3005.0
-----
Volvo's mean price in the dataset is : 4993.0
-----
Renault's mean price in the dataset is : 2496.0
-----
Fiat's mean price in the dataset is : 2837.0
-----
Smart's mean price in the dataset is : 3596.0
-----
Hyundai's mean price in the dataset is : 5411.0
-----
Peugeot's mean price in the dataset is : 3114.0
-----
Toyota's mean price in the dataset is : 5167.0
-----
Bmw's mean price in the dat

`sonstige_auto` is German for "Other cars" which is why it's data is rather interesting. 

`mini` seem to hold their value well in Germany.

Generally the premium manufactures sell for a high price which is to be expected.

## Comparing mean price to mileage



In [45]:
brand_mean_mileage = make_mean(top_brands, autos, 'brand', 'odometer_km')

bmp_s = pd.Series(brand_mean_price)
bmm_s = pd.Series(brand_mean_mileage)

means_df = pd.DataFrame(bmp_s, columns=['mean_price'])
means_df['mean_mileage'] = bmm_s

print(means_df)

                  mean_price   mean_mileage
audi             9380.718548  129245.400298
bmw              8381.677306  132695.320149
citroen          3796.262673  119554.531490
fiat             2836.873631  116950.294861
ford             3779.269076  124277.108434
hyundai          5411.075431  106885.775862
mazda            4129.774788  124553.824363
mercedes_benz    8672.654241  131025.669643
mini            10639.450980   88308.823529
nissan           4756.659634  118326.300985
opel             3005.496077  129384.429692
peugeot          3113.860549  127127.890173
renault          2496.070577  128281.393217
seat             4433.419622  121536.643026
skoda            6409.609724  110906.701708
smart            3596.402736   99734.042553
sonstige_autos  12784.556561   90395.927602
toyota           5167.091062  115944.350759
volkswagen       5436.950097  128799.877539
volvo            4993.208038  138581.560284
